In [1]:
%load_ext jupyter_black
import requests
import json
import pandas as pd
from tqdm.notebook import tqdm
import yaml
import time

In [5]:
with open("secrets.yaml") as secret:
    secret = yaml.load(secret, Loader=yaml.Loader)
BEARER_TOKEN = secret["token_essential"]
# define search twitter function
def search_twitter(
    query, tweet_fields, next_token=None, count=0, bearer_token=BEARER_TOKEN
):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    if count == 0:

        url = "https://api.twitter.com/2/tweets/search/recent?max_results=100&query={}&tweet.fields={}".format(
            query, tweet_fields
        )
    else:
        url = "https://api.twitter.com/2/tweets/search/recent?max_results=100&query={}&tweet.fields={}&pagination_token={}".format(
            query, tweet_fields, next_token
        )
    response = requests.request("GET", url, headers=headers)

    # print(f'Status: {response.status_code} and count: {count}')

    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


# search term
query = "economy"
tweet_fields = "created_at,id,text"

# twitter api call
main_df = pd.DataFrame()
COUNT = 0
NEXT_TOKEN = None
for i in tqdm(range(10)):
    json_response = search_twitter(
        query,
        next_token=NEXT_TOKEN,
        tweet_fields=tweet_fields,
        count=COUNT,
        bearer_token=BEARER_TOKEN,
    )
    NEXT_TOKEN = json_response["meta"]["next_token"]
    df = pd.DataFrame(json_response["data"])
    main_df = pd.concat([main_df, df], axis=0, ignore_index=True)
    COUNT = COUNT + 1
    time.sleep(3)

  0%|          | 0/10 [00:00<?, ?it/s]